In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import torch.nn.functional as F 
import pickle
import numpy as np
import pandas as pd
import os
import glob
import lzma
import cv2

2024-11-15 15:30:15.844624: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731681015.858415  193765 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731681015.862128  193765 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 15:30:15.878423: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#set the device to cuda if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
# here we're going to create a list with the colors we followed for each record in RGB 
# 0 - red
record_colors = [(255,0,0),(0,0,255),(0, 255, 255)]

In [ ]:
data_images=[]
data_labels=[]
data_colors=[]

#for each record we're going to create a list of snapshots 
for record in glob.glob("*.npz"):   
    try:
        with lzma.open(record, "rb") as file:
            data = pickle.load(file)
            print("Read", len(data), "snapshots")
            # take color from number of file in record_colors
            color = record_colors[int(record.replace('.npz', '').split('_')[1])]
            #normalize color value RGB
            color = [c / 255 for c in color]
            #  for each snapshot we're goint to have the image, the controls on that image and the color of that snapshot, each record will have always th same colors
            for snapshot in data:
                data_images.append(snapshot.image)
                data_labels.append(snapshot.current_controls)
                data_colors.append(color)
    except EOFError:
        print("Error: Compressed file ended before the end-of-stream marker was reached.")
    except Exception as e:
        print(f"An error occurred: {e}")
    
    print("Total snapshots read:", len(data_images))


package_folder: /home/francisco/.conda/envs/myenv/lib/python3.12/site-packages/ursina
asset_folder: /home/francisco/.conda/envs/myenv/lib/python3.12/site-packages


which: no blender in (/home/francisco/.conda/envs/myenv/bin:/usr/condabin:/home/francisco/.local/bin:/home/francisco/bin:/usr/local/bin:/usr/local/sbin:/usr/bin:/usr/sbin)


Read 1003 snapshots
Total snapshots read: 1003
Read 1009 snapshots
Total snapshots read: 2012
An error occurred: No module named 'sensing_message'
Total snapshots read: 2012


In [5]:
from collections import Counter
##up down left right
label_counts = Counter([tuple(label) for label in data_labels])
print("Label distribution:", label_counts)


Label distribution: Counter({(1, 0, 0, 0): 619, (1, 0, 0, 1): 324, (1, 0, 1, 0): 319, (0, 0, 0, 1): 272, (0, 0, 1, 0): 208, (0, 1, 1, 0): 105, (0, 0, 0, 0): 83, (0, 1, 0, 1): 62, (0, 1, 0, 0): 11, (1, 0, 1, 1): 4, (0, 0, 1, 1): 4, (1, 1, 1, 0): 1})


In [6]:
for i in data_images:
    #resize with cv2
    i = cv2.resize(i, (640, 512))


In [9]:

data_images_augmented = []
data_labels_augmented = []
data_colors_augmented = []

for image, label, color in zip(data_images, data_labels, data_colors):
    data_images_augmented.append(image)
    #   Forward - Backward - Left - Right
    data_labels_augmented.append(label)
    data_colors_augmented.append(color)
    
    flipped_image = np.flip(image, axis=1)
    flipped_label = (label[0], label[1], label[3], label[2])

    data_images_augmented.append(flipped_image)
    data_labels_augmented.append(flipped_label)
    data_colors_augmented.append(color)

#add that to the data_images,data_labels and data_colors the augmented data
data_images += data_images_augmented
data_labels += data_labels_augmented
data_colors += data_colors_augmented


In [10]:
#Split the data into training and testing
X_train, X_test, y_train, y_test, color_train, color_test = train_test_split(data_images, data_labels, data_colors, test_size=0.2, random_state=42)

In [11]:
class CustomDataset(Dataset):
    def __init__(self, X, y, c, transform=None):
        self.X = X
        self.y = y
        self.c = c
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        image = self.X[idx]
        label = self.y[idx]
        color = self.c[idx]
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.float32), torch.tensor(color, dtype=torch.float32)


train_dataset = CustomDataset(X_train, y_train, color_train)

In [12]:
#Initiate the dataloader
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [13]:
#Initiate the model
from scripts.models import CNN, CNNColor, ConcatModel
model = ConcatModel(CNN(3), CNNColor(1)).to(device)

/home/francisco/.conda/envs/myenv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/francisco/.conda/envs/myenv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
#Loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [16]:
# Assuming train_loader is defined
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for i, (images, labels, colors) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        colors = colors.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, torch.cat((labels, colors), 1))
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

ValueError: At least one stride in the given numpy array is negative, and tensors with negative strides are not currently supported. (You can probably work around this by making a copy of your array  with array.copy().) 

In [ ]:
# Save the model
torch.save(model.state_dict(), "model.pth")